In [3]:
%%python - AIM-1 AIM-2 AIM-3 AIM-6 AIM-7 AIM-8 AIM-9 AIM-10

import argparse
import os
import json
from jira import JIRA
import openai

def load_openai_api_key(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        openai.api_key = data['openai']
        

def load_jira_api_key(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        jira_key = data['jira']
        return jira_key
    
def summarize_jira_issue(description):
    prompt = f"""
    You are writing knowledgebase article content for a customer of a system,\
    who needs a brief overview of the fixed problem, the provided solution; and if applicable, any action the customer must take.\
    Write knowledgebase article content based on the following Jira issue description,\ 
    summarized in three bullet points, formatted as follows:\
    1) Problem:\
    2) Solution:\
    3) Required Customer Action:\
    If there is no coherent problem or solution information in the Jira issue description or the description is empty, then write 'No information available!'\
    If there is no solution mentioned in the Jira issue description, write 'No solution provided.' in the Solution bullet point.\
    If no explicit customer action mentioned in the Jira issue description, identify the customer action based on the solution or workarounds or actions mentioned in the Jira issue description.\
    For example, 'Restarting system is the only way for the update to take effect' indicates the required customer action of system restart.\
    For example, a solution like 'A new button was added to enable operators to manually perform the action' indicates the required customer action of informing customer's operators to click the button to manually perform the action.\
    The customer action should preferably be in unbiased and gender-neutral language. Only if absolutely required, use 'they', 'them' and 'their' pronouns for the customer.\
    If no customer action can be identified in the Jira issue description, write 'No customer action required.' in the Required Customer Action bullet point.\
    \\n\\nJira issue description: {description}"""
    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
    )
    
    summary = response.choices[0].message["content"]
    return summary.strip()

def main():
    
    load_openai_api_key('api_keys.json')
    jira_key = load_jira_api_key('api_keys.json')
    
    parser = argparse.ArgumentParser(description="Summarize Jira issues.")
    parser.add_argument("issue_ids", metavar="N", type=str, nargs="+", help="Jira issue IDs")
    args = parser.parse_args()

    jira = JIRA(server="https://aimugdha.atlassian.net", basic_auth=('vmugdha.ai@gmail.com', jira_key))
    issue_ids = args.issue_ids

    output_dir = "KBA_Output"
    os.makedirs(output_dir, exist_ok=True)

    for issue_id in issue_ids:
        issue = jira.issue(issue_id)
        summary = summarize_jira_issue(issue.fields.description)
        with open(f"{output_dir}/{issue_id}.txt", "w") as f:
            f.write(str(summary))

    print(f"Successfully processed {len(issue_ids)} Jira issues.")

if __name__ == "__main__":
    main()

Successfully processed 8 Jira issues.
